In [3]:
import pandas as pd
import librosa
import pyworld
import numpy as np

# Pre-Processing
WORLD Encoding

In [1]:
# import librosa
# import pyworld
# def load_wavs(wav_dir, sr):
#     wavs = list()
#     for file in os.listdir(wav_dir):
#         file_path = os.path.join(wav_dir, file)
#         wav, _ = librosa.load(file_path, sr=sr, mono=True)
#         # wav = wav.astype(np.float64)
#         wavs.append(wav)
#     return wavs

# def world_decompose(wav, fs, frame_period=5.0):
#     # Decompose speech signal into f0, spectral envelope and aperiodicity using WORLD
#     wav = wav.astype(np.float64)
#     f0, timeaxis = pyworld.harvest(
#         wav, fs, frame_period=frame_period, f0_floor=71.0, f0_ceil=800.0)

#     # Finding Spectogram
#     sp = pyworld.cheaptrick(wav, f0, timeaxis, fs)

#     # Finding aperiodicity
#     ap = pyworld.d4c(wav, f0, timeaxis, fs)

#     # Use this in Ipython to see plot
#     # librosa.display.specshow(np.log(sp).T,
#     #                          sr=fs,
#     #                          hop_length=int(0.001 * fs * frame_period),
#     #                          x_axis="time",
#     #                          y_axis="linear",
#     #                          cmap="magma")
#     # colorbar()
#     return f0, timeaxis, sp, ap


# def world_encode_spectral_envelop(sp, fs, dim=24):
#     # Get Mel-Cepstral coefficients (MCEPs)
#     # sp = sp.astype(np.float64)
#     coded_sp = pyworld.code_spectral_envelope(sp, fs, dim)
#     return coded_sp


# def world_encode_data(wave, fs, frame_period=5.0, coded_dim=24):
#     f0s = list()
#     timeaxes = list()
#     sps = list()
#     aps = list()
#     coded_sps = list()
#     for wav in wave:
#         f0, timeaxis, sp, ap = world_decompose(wav=wav,
#                                                fs=fs,
#                                                frame_period=frame_period)
#         coded_sp = world_encode_spectral_envelop(sp=sp, fs=fs, dim=coded_dim)
#         f0s.append(f0)
#         timeaxes.append(timeaxis)
#         sps.append(sp)
#         aps.append(ap)
#         coded_sps.append(coded_sp)
#     return f0s, timeaxes, sps, aps, coded_sps


# def logf0_statistics(f0s):
#     # Note: np.ma.log() calculating log on masked array (for incomplete or invalid entries in array)
#     log_f0s_concatenated = np.ma.log(np.concatenate(f0s))
#     log_f0s_mean = log_f0s_concatenated.mean()
#     log_f0s_std = log_f0s_concatenated.std()
#     return log_f0s_mean, log_f0s_std


# def transpose_in_list(lst):
#     transposed_lst = list()
#     for array in lst:
#         transposed_lst.append(array.T)
#     return transposed_lst


# def coded_sps_normalization_fit_transform(coded_sps):
#     coded_sps_concatenated = np.concatenate(coded_sps, axis=1)
#     coded_sps_mean = np.mean(coded_sps_concatenated, axis=1, keepdims=True)
#     coded_sps_std = np.std(coded_sps_concatenated, axis=1, keepdims=True)
#     coded_sps_normalized = list()
#     for coded_sp in coded_sps:
#         coded_sps_normalized.append(
#             (coded_sp - coded_sps_mean) / coded_sps_std)
#     return coded_sps_normalized, coded_sps_mean, coded_sps_std

In [ ]:
# from scipy.signal import get_window
# from librosa.filtersx import mel
# from scipy import signal

# def psnr_np(pred, target, target_top=True, top=1e4):
#     """
#     Peak Signal-to-Noise Ratio (but not really) - NumPy version
#     Since spectrograms values are unbounded, this function uses `max(target)` as the maximum possible value by default.
#     """
#     if target_top:
#         top = target.max()
#     return 20 * np.log10(top / np.sqrt(((pred - target) ** 2).mean()))

# def pySTFT(x, fft_length=2048, hop_length=1024):
    
#     x = np.pad(x, int(fft_length//2), mode='reflect')
    
#     noverlap = fft_length - hop_length
#     shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)
#     strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])
#     result = np.lib.stride_tricks.as_strided(x, shape=shape,
#                                              strides=strides)
    
#     fft_window = get_window('hann', fft_length, fftbins=True)
#     result = np.fft.rfft(fft_window * result, n=fft_length).T
    
#     return np.abs(result) 

# def butter_highpass(cutoff, fs, order=5):
#     nyq = 0.5 * fs
#     normal_cutoff = cutoff / nyq
#     b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
#     return b, a
# def get_embeddings(x):
#     D = pySTFT(x).T
#     mel_basis = mel(sr = 16000, n_fft = 2048, fmin=90, fmax=7600, n_mels=80).T
#     min_level = np.exp(-100 / 20 * np.log(10))
#     b, a = butter_highpass(30, 16000, order=5)

#     # Convert to mel and normalize
#     D_mel = np.dot(D, mel_basis)
#     D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
#     S = np.clip((D_db + 100) / 100, 0, 1)   
     


WORLD DATA EXTRACTION

In [26]:
rootDir = './wavs'
dirName, subdirList, _ = next(os.walk(rootDir))
df=pd.DataFrame()
rootdirs = []
subdirs=[]
filenames=[]
pathnames = []
audio_files=[]
sp_s = []
f0_s = []
ap_s = []

for subdir in subdirList:
    _, _, fileList = next(os.walk(os.path.join(dirName,subdir)))
    for filename in fileList:
        rootdirs.append(rootDir)
        subdirs.append(subdir)
        filenames.append(filename)
        path = os.path.join(dirName, subdir, filename)
        pathnames.append(path)
        wav, _ = librosa.load(path, sr=44100, mono=True) # TODO: consider changing sample rates
        audio_files.append(wav)
        wav = wav.astype(np.float64)   
        f0, timeaxis = pyworld.harvest(
        wav, fs=44100, frame_period=5, f0_floor=71.0, f0_ceil=800.0) # frame_period of 5 implies 2048 block
        f0_s.append(f0)
        # Finding Spectogram
        sp = pyworld.cheaptrick(wav, f0, timeaxis, fs=44100)
        sp_s.append(sp)
        # Finding aperiodicity
        ap = pyworld.d4c(wav, f0, timeaxis, fs=44100)
        ap_s.append(ap)


df['rootdir']=rootdirs
df['subdir'] = subdirs
df['filename'] = filenames
df['path'] = pathnames
df['audio'] = audio_files

df['f0'] = f0_s
df['sp'] = sp_s
df['ap'] = sp_s

### METADATA EXTRACTION

Get mel spectrogram

Note:
the input file names need to be encoded as (letter)XXXX where xxxx is an integer and (letter) is an alphabet

In [27]:
from make_spect import get_spect
get_spect('./wavs','./processed_data/spmels')

Found directory: ./wavs
p225
p226
p227
p228


Get speaker embeddings

In [28]:
import numpy as np
import torch
from model_bl import D_VECTOR
from collections import OrderedDict
C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cuda()
c_checkpoint = torch.load('3000000-BL.ckpt')
new_state_dict = OrderedDict()
for key, val in c_checkpoint['model_b'].items():
    new_key = key[7:]
    new_state_dict[new_key] = val
C.load_state_dict(new_state_dict)

len_crop = 128
rootDir='./processed_data/spmels'
writeDir = './processed_data/embeddings'
dirName, subdirList, _ = next(os.walk(rootDir))
for subdir in subdirList:
    _, _, fileList = next(os.walk(os.path.join(dirName,subdir)))
    embs = []
    for filename in fileList:
        if not os.path.exists(os.path.join(writeDir, subdir)):
            os.makedirs(os.path.join(writeDir, subdir))
        _,_, fileList = next(os.walk(os.path.join(dirName,subdir)))
        x = np.load(os.path.join(dirName, subdir, filename))
        left = np.random.randint(0, x.shape[0]-len_crop)
        melsp = torch.from_numpy(x[np.newaxis, left:left+len_crop, :]).cuda()
        emb = C(melsp).detach().squeeze().cpu().numpy()
        embs.append(emb)
        # np.save(os.path.join(writeDir, subdir, filename[:-4]), emb.astype(np.float32), allow_pickle=False)    
    np.save(os.path.join(writeDir, subdir, filename[:-4]), np.mean(embs, axis=0).astype(np.float32), allow_pickle=False) 


In [29]:
metadf = pd.DataFrame()
rootDir = './wavs'
spmelDir = './processed_data/spmels'
dirName, subdirList, _ = next(os.walk(spmelDir))
metadf['subdir'] = subdirList
embDir = './processed_data/embeddings'
embeddings=[]
for subdir in subdirList:
    _, _, fileList = next(os.walk(os.path.join(embDir,subdir)))
    x = np.load(os.path.join(embDir, subdir, fileList[0]))
    embeddings.append(x)

metadf['embeddings'] = embeddings


In [30]:
metadf

,subdir,embeddings
0,p228,"[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
1,p225,"[-0.018470041, 0.019759562, 0.021973787, 0.067..."
2,p226,"[0.04604619, -0.019872962, 0.030422132, -0.038..."
3,p227,"[0.06895803, -0.011125765, 0.008205971, -0.001..."


In [33]:
df1=df.merge(metadf, 'inner',on='subdir')
df1

,rootdir,subdir,filename,path,audio,f0,sp,ap,embeddings
0,./wavs,p228,p228_024.wav,./wavs/p228/p228_024.wav,"[0.0029742254, 0.0034336508, 0.0032673946, 0.0...","[431.50114345396275, 423.915993425333, 411.875...","[[4.363340813554503e-05, 4.360484986123105e-05...","[[4.363340813554503e-05, 4.360484986123105e-05...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
1,./wavs,p228,p228_016.wav,./wavs/p228/p228_016.wav,"[-0.0031878096, -0.0039010416, -0.0041681095, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 215.06789720603257, ...","[[1.620186893150466e-05, 1.6200625522465935e-0...","[[1.620186893150466e-05, 1.6200625522465935e-0...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
2,./wavs,p228,p228_023.wav,./wavs/p228/p228_023.wav,"[-0.00088880287, -0.0010265178, -0.0010094485,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 134.3...","[[1.404731254305884e-05, 1.4043958535403253e-0...","[[1.404731254305884e-05, 1.4043958535403253e-0...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
3,./wavs,p228,p228_008.wav,./wavs/p228/p228_008.wav,"[0.0023408053, 0.002647261, 0.0025867063, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[5.286216707935018e-05, 5.291916137681443e-05...","[[5.286216707935018e-05, 5.291916137681443e-05...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
4,./wavs,p228,p228_019.wav,./wavs/p228/p228_019.wav,"[0.0027019747, 0.0031474838, 0.003122318, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[3.968113806677623e-05, 3.971109467313459e-05...","[[3.968113806677623e-05, 3.971109467313459e-05...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
5,./wavs,p228,p228_021.wav,./wavs/p228/p228_021.wav,"[0.0004937804, 0.0006738206, 0.0008547033, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[5.9466123977266e-05, 5.945200341313279e-05, ...","[[5.9466123977266e-05, 5.945200341313279e-05, ...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
6,./wavs,p228,p228_005.wav,./wavs/p228/p228_005.wav,"[0.0019450915, 0.002293077, 0.0022817107, 0.00...","[77.12731736300209, 78.70471568402388, 80.3352...","[[0.002132257351339337, 0.0022370689570899225,...","[[0.002132257351339337, 0.0022370689570899225,...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
7,./wavs,p228,p228_003.wav,./wavs/p228/p228_003.wav,"[0.0011369577, 0.0013415589, 0.0013888823, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1.3113334590531052e-05, 1.3116559646890245e-...","[[1.3113334590531052e-05, 1.3116559646890245e-...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
8,./wavs,p228,p228_011.wav,./wavs/p228/p228_011.wav,"[-0.00026696932, -0.00015485777, 2.9849007e-05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 266.54767529010...","[[3.8225308017049225e-06, 3.825157148581609e-0...","[[3.8225308017049225e-06, 3.825157148581609e-0...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."
9,./wavs,p228,p228_022.wav,./wavs/p228/p228_022.wav,"[0.0011427994, 0.001313226, 0.0012786873, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 288.36309583712...","[[2.756140143122461e-05, 2.7565413101544567e-0...","[[2.756140143122461e-05, 2.7565413101544567e-0...","[-0.04824982, -0.014440345, 0.05722511, 0.0318..."


In [34]:
df1.to_pickle('./world_train.pkl')

In [22]:
df = pd.read_pickle('./world_train.pkl')

In [ ]:
# #TODO: Consider up/down sampling to a fixed fs
# # f0, timeaxis, sp, ap = world_decompose(wavs[0], 16000, 5.0) # WAVS IS A LIST OF MONO AUDIO DATA 
# # coded_sp = world_encode_spectral_envelop(sp, 16000, 24)
# f0s, timeaxes, sps, aps, coded_sps = world_encode_data(wavs, 16000, 5, 24)
# coded_sps_transposed = transpose_in_list(lst=coded_sps)
# # print(coded_sps_transposed)

# coded_sps_norm, coded_sps_mean, coded_sps_std = coded_sps_normalization_fit_transform(
#     coded_sps=coded_sps_transposed)

### Style Embedding Extraction

make_metadata.py

In [12]:
import numpy as np
import torch
from model_bl import D_VECTOR
from collections import OrderedDict
C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cuda()
c_checkpoint = torch.load('3000000-BL.ckpt')
new_state_dict = OrderedDict()
for key, val in c_checkpoint['model_b'].items():
    new_key = key[7:]
    new_state_dict[new_key] = val
C.load_state_dict(new_state_dict)

len_crop = 128
rootDir='./processed_data/spmels'
writeDir = './processed_data/embeddings'
dirName, subdirList, _ = next(os.walk(rootDir))
for subdir in subdirList:
    _, _, fileList = next(os.walk(os.path.join(dirName,subdir)))
    embs = []
    for filename in fileList:
        if not os.path.exists(os.path.join(writeDir, subdir)):
            os.makedirs(os.path.join(writeDir, subdir))
        _,_, fileList = next(os.walk(os.path.join(dirName,subdir)))
        x = np.load(os.path.join(dirName, subdir, filename))
        left = np.random.randint(0, x.shape[0]-len_crop)
        melsp = torch.from_numpy(x[np.newaxis, left:left+len_crop, :]).cuda()
        emb = C(melsp).detach().squeeze().cpu().numpy()
        embs.append(emb)
        # np.save(os.path.join(writeDir, subdir, filename[:-4]), emb.astype(np.float32), allow_pickle=False)    
    np.save(os.path.join(writeDir, subdir, filename[:-4]), np.mean(embs, axis=0).astype(np.float32), allow_pickle=False) 


### Dataloader testing

In [52]:
from data_loader import get_loader
vcc_loader = get_loader('./spmel', 3, 128)
import pandas as pd
DF=pd.DataFrame(next(iter(vcc_loader)))

Finished loading the dataset...


/home/vkalbag/miniconda3/lib/python3.9/site-packages/pandas/core/internals/construction.py:576: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  values = np.array([convert(v) for v in values])
/home/vkalbag/miniconda3/lib/python3.9/site-packages/pandas/core/internals/construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v 

In [53]:
next(iter(vcc_loader))[0].shape

torch.Size([3, 128, 80])

In [54]:
next(iter(vcc_loader))[1].shape

torch.Size([3, 256])

First element of the dataloader is the mel spectrogram (to be replaced with sp) and the second is the speaker embeddings

# Dataloader and dataset class

In [1]:
import pandas as pd
import librosa
import pyworld
import numpy as np

In [2]:
from torch.utils import data
import torch
import numpy as np
import pickle 
import os    
       
from multiprocessing import Process, Manager 

class SpeechVocalDataset(data.Dataset):
    def __init__(self,root_dir='./processed_data', len_crop=128):
        self.root_dir = root_dir
        self.len_crop = len_crop
        self.step = 10
        
        metaname = os.path.join(self.root_dir, "world_train.pkl") # TODO: Rather than using a single pickle file, use one pickle file for each row
        self.df=pd.read_pickle(metaname)
        self.num_tokens=len(self.df)

    def __getitem__(self,index):
        row = self.df.iloc[index]
        tmp_sp = row[6]
        embedding = row[8]

        if tmp_sp.shape[0] < self.len_crop:
            len_pad = self.len_crop - tmp_sp.shape[0]
            sp = np.pad(tmp_sp, ((0,len_pad),(0,0)), 'constant')
        elif tmp_sp.shape[0] > self.len_crop:
            left = np.random.randint(tmp_sp.shape[0]-self.len_crop)
            sp = tmp_sp[left:left+self.len_crop, :]
        else:
            sp = tmp_sp
        
        return sp.astype(np.float64),embedding.astype(np.float64)
    
    def __len__(self):
        """Return the number of spkrs."""
        return self.num_tokens
        

In [3]:
dataset = SpeechVocalDataset('./processed_data', 128)
    
worker_init_fn = lambda x: np.random.seed((torch.initial_seed()) % (2**32))
data_loader = data.DataLoader(dataset=dataset,
                                batch_size=16,
                                shuffle=True,
                                num_workers=0,
                                drop_last=True,
                                worker_init_fn=worker_init_fn)

## Training

In [1]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('-lambda_cd', help="lambda_cd", type=int, required=False, default=1)
parser.add_argument('-dim_neck', help="", type=int, required=False, default=16)
parser.add_argument('-dim_emb', help="", type=int, required=False, default=256)
parser.add_argument('-dim_pre', help="", type=int, required=False, default=512)
parser.add_argument('-freq',help='',type=int, required=False, default=16)
parser.add_argument('-data_dir',help='',type=str, required=False, default='./processed_data')
parser.add_argument('-batch_size', help='', type=int, required=False, default=2)
parser.add_argument('-num_iters',help='',type=int, required=False, default=2000)
parser.add_argument('-len_crop',help='',type=int,required=False,default=128)
parser.add_argument('-log_step',help='',type=int,required=False,default=10)
config = parser.parse_args([])

10

In [5]:
import os
# import argparse
from solver_encoder import Solver
from data_loader import get_loader
from torch.backends import cudnn

cudnn.benchmark = True

# Data loader.
worker_init_fn = lambda x: np.random.seed((torch.initial_seed()) % (2**32))
vcc_loader = data.DataLoader(dataset=dataset,
                                batch_size=16,
                                shuffle=True,
                                num_workers=0,
                                drop_last=True,
                                worker_init_fn=worker_init_fn)
solver = Solver(vcc_loader, config)

solver.train()

Setting model to GPU
Start training...
Setting inputs to GPU


/home/vkalbag/projects/singing-voice-conversion/autosvc-vedant/solver_encoder.py:91: UserWarning: Using a target size (torch.Size([16, 1, 128, 1025])) that is different to the input size (torch.Size([16, 128, 1025])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  g_loss_id = F.mse_loss(x_real, x_identic)
/home/vkalbag/projects/singing-voice-conversion/autosvc-vedant/solver_encoder.py:92: UserWarning: Using a target size (torch.Size([16, 1, 128, 1025])) that is different to the input size (torch.Size([16, 128, 1025])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  g_loss_id_psnt = F.mse_loss(x_real, x_identic_psnt)


Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Elapsed [0:00:03], Iteration [10/1000000], G/loss_id: 0.0453, G/loss_id_psnt: 1.0361, G/loss_cd: 0.0776
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Setting inputs to GPU
Elapsed [0:00:06], Iteration [20/1000000], G/loss_id: 0.0629, G/loss_id_psnt: 1.0250, G/loss_cd: 0.0553
Setting inputs to GPU


KeyboardInterrupt: 